In [2]:
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import random
import json
import os
import tqdm
import time
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import random
from torch.utils.data import random_split
from utils import feature_calculation,lable_making
import json

In [3]:

# 加载数据
natural_train = torch.load('/root/autodl-tmp/Python projects/Dataset/expand/mfcc/natural_train_MFCC.pt')
ep_train = torch.load('/root/autodl-tmp/Python projects/Dataset/expand/mfcc/ep_train_MFCC.pt')
ss_train = torch.load('/root/autodl-tmp/Python projects/Dataset/expand/mfcc/ss_train_MFCC.pt')
natural_test = torch.load('/root/autodl-tmp/Python projects/Dataset/expand/mfcc/natural_test_MFCC.pt')
ep_test = torch.load('/root/autodl-tmp/Python projects/Dataset/expand/mfcc/ep_test_MFCC.pt')
ss_test = torch.load('/root/autodl-tmp/Python projects/Dataset/expand/mfcc/ss_test_MFCC.pt')
print(natural_train.shape)
print(ep_train.shape)
print(ss_train.shape)
print(natural_test.shape)
print(ep_test.shape)
print(ss_test.shape)

torch.Size([43295, 3, 40, 72])
torch.Size([44000, 3, 40, 72])
torch.Size([42490, 3, 40, 72])
torch.Size([10986, 3, 40, 72])
torch.Size([11230, 3, 40, 72])
torch.Size([9950, 3, 40, 72])


In [ ]:
print()

In [ ]:
def dataloader(natural_train_path_0,natural_train_path_1,ep_train_path,ss_train_path,
                batch_size=64,validation_percentage = 0.1):
    loaded_natural_train_0 = torch.load(natural_train_path_0)
    # loaded_natural_train_0=torch.tensor(loaded_natural_train_0)
    if natural_train_path_1:
        loaded_natural_train_1 = torch.load(natural_train_path_1)
        # loaded_natural_train_1=torch.tensor(loaded_natural_train_1)
        loaded_natural_train = torch.cat((loaded_natural_train_0, loaded_natural_train_1), dim=0)
        del loaded_natural_train_0,loaded_natural_train_1
    else:
        loaded_natural_train=loaded_natural_train_0
        del loaded_natural_train_0
    loaded_ep_train = torch.load(ep_train_path)
    # loaded_ep_train=torch.tensor(loaded_ep_train)
    loaded_ss_train = torch.load(ss_train_path)

    # loaded_ss_train=torch.tensor(loaded_ss_train)
    
    loaded_natural_train=torch.squeeze(loaded_natural_train)
    loaded_ep_train=torch.squeeze(loaded_ep_train)
    loaded_ss_train=torch.squeeze(loaded_ss_train)
    #划分训练集与验证集
    natural_val_size = int(validation_percentage * loaded_natural_train.shape[0])
    natural_train_size = loaded_natural_train.shape[0] - natural_val_size
    ep_val_size = int(validation_percentage * loaded_ep_train.shape[0])
    ep_train_size = loaded_ep_train.shape[0] - ep_val_size
    ss_val_size = int(validation_percentage * loaded_ss_train.shape[0])
    ss_train_size = loaded_ss_train.shape[0] - ss_val_size

    natural_train=loaded_natural_train[:natural_train_size]
    ep_train=loaded_ep_train[:ep_train_size]
    ss_train=loaded_ss_train[:ss_train_size]
    #标签编码 0天然地震 1爆破 2塌陷
    zero_tensor_train = lable_making(natural_train,0)
    one_tensor_train = lable_making(ep_train,1)
    two_tensor_train = lable_making(ss_train,2)

    natural_valid=loaded_natural_train[natural_val_size:]
    ep_valid=loaded_ep_train[ep_val_size:]
    ss_valid=loaded_ss_train[ss_val_size:]
    #标签编码 0天然地震 1爆破 2塌陷
    zero_tensor_valid = lable_making(natural_valid,0)
    one_tensor_valid = lable_making(ep_valid,1)
    two_tensor_valid= lable_making(ss_valid,2)
    del loaded_natural_train,loaded_ep_train,loaded_ss_train
    # 使用torch.cat在维度0上合并两个张量  
    train_tensor = torch.cat((natural_train, ep_train,ss_train), dim=0)
    del natural_train,ep_train,ss_train
    train_lable_tensor=torch.cat((zero_tensor_train, one_tensor_train,two_tensor_train), dim=0)
    del zero_tensor_train,one_tensor_train,two_tensor_train

    valid_tensor = torch.cat((natural_valid, ep_valid,ss_valid), dim=0)
    del natural_valid,ep_valid,ss_valid
    valid_lable_tensor=torch.cat((zero_tensor_valid, one_tensor_valid,two_tensor_valid), dim=0)
    del zero_tensor_valid,one_tensor_valid,two_tensor_valid

    # 创建TensorDataset并将图像数据和标签合并
    train_dataset = TensorDataset(train_tensor, train_lable_tensor)
    valid_dataset = TensorDataset(valid_tensor, valid_lable_tensor)

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)

    return train_loader,valid_loader  

In [ ]:
train_loader,val_loader =dataloader(natural_train_path_0=base_path+'/mfcc/natural_train_MFCC.pt',
                                            natural_train_path_1=None,
                                            ep_train_path=base_path+'/mfcc/ep_train_MFCC.pt',
                                            ss_train_path=base_path+'/mfcc/ss_train_MFCC.pt',
                                            batch_size=32,validation_percentage=0.1)